In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%matplotlib inline

In [3]:
import os

In [4]:
pwd

'/Users/obinnadinneya/Desktop/OBI_PORTFOLIO'

In [5]:
#list all files in a directory 
#you can use a for loop to open all files in the directory too
os.listdir()

['GERMAN CREDITABILITY EDA AND PREDICTION  .ipynb',
 'Support-Vector-Machines.ipynb',
 '.DS_Store',
 'Recommender System for movie dataset.ipynb',
 'European soccer database EDA.ipynb',
 'README.md',
 'Airbnb_Sydney_SqlProject.ipynb',
 '.ipynb_checkpoints',
 'REAL ESTATE EDA AND PRICE PREDICTION.ipynb',
 '.git',
 'Exploratory Analysis on data science job salaries 2020-2023.ipynb',
 'WebscrappingProject.ipynb']

In [6]:
housing_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
housing_df.head()

In [ ]:
#print the dimension/shape of the dataframe
housing_df.shape

In [ ]:
housing_df.info()

In [ ]:
#removing the ID column
housing_df.drop("Id",axis = 1 ,inplace = True)

In [ ]:
plt.figure(figsize=(13,6))
sns.heatmap(housing_df.isnull(),yticklabels=False,cmap='viridis',cbar=False)

In [ ]:
housing_df.isnull().sum()

In [ ]:
null_df = housing_df.isnull().sum()*100/len(housing_df)

In [ ]:
null_df

In [ ]:
#print columns that have values > 25%
#this shows the less influential variables 
null_df[null_df.values > 25].index

In [ ]:
housing_df.drop(['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], axis =1,
               inplace = True)

In [ ]:
null_df[(null_df.values <= 25) & (null_df.values > 0)]

In [ ]:
housing_df['LotFrontage'].median()

In [ ]:
#since there are still quite a bit of missing vaues in lotfrontage 
#we can fill it in with the median value 

housing_df['LotFrontage'].fillna(69.0,inplace = True)

In [ ]:
#returning location where msvnrarea is null as a data frame but for only
#MasVnrArea','MasVnrType columns

housing_df.loc[housing_df.MasVnrArea.isnull(),['MasVnrArea','MasVnrType']]

In [ ]:
housing_df[['MasVnrArea','MasVnrType']].head()

In [ ]:
#now that we know where the area is null the type is also null
#fill in the values 
housing_df.MasVnrArea.fillna(0,inplace = True)
housing_df.MasVnrType.fillna('None', inplace = True)

In [ ]:
housing_df[['GarageYrBlt','GarageFinish','GarageQual','GarageCond']]

In [ ]:
housing_df.loc[housing_df.GarageType.isnull(),['GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond']]

Considering that the garage is empty for these houses 

In [ ]:
housing_df.GarageType.fillna('None', inplace=True)
housing_df.GarageYrBlt.fillna(0, inplace=True)
housing_df.GarageFinish.fillna('None', inplace=True)
housing_df.GarageQual.fillna('None', inplace=True)
housing_df.GarageCond.fillna('None', inplace=True)

In [ ]:
#removing all rows with Na, NaN , null values
housing_df.dropna(inplace = True)

In [ ]:
#check if there are any na values left 
#should return an empty series 
null_df = housing_df.isnull().sum()
null_df[null_df.values > 0]

In [ ]:
sns.heatmap(housing_df.isnull(),cmap='viridis',cbar=False,yticklabels=False)

In [ ]:
#we can ask for a statistical summary of each column and specify
#the percentile
#Adding .T allows us to view the columns as rows and vice versa 
housing_df.describe([0,0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99,1]).T


### To remove outliers which are +-3 standard deviations away from the mean you would have to get rid of anything above the 99 percentile since we only have positive values 

In [ ]:
#excluded object datatypes and printed, assigned a list of non- object
#columns
number_cols = housing_df.select_dtypes(exclude= 'object').columns
number_cols

In [ ]:
np.percentile(housing_df['MSSubClass'],99)

### In order to exclude anything higher than 99% of each int/float column then what we have to do is change the value to be the 99%tile value using a for loop 

In [ ]:
type(number_cols)

In [ ]:
num_cols = list(number_cols)

### since the variable above is an index not a list to iterate through we have to convert it to a list when we are calling the for loop

In [ ]:
num_cols

In [ ]:
for i in num_cols:
    housing_df.loc[housing_df[i] >= np.percentile(housing_df[i], 99),i] = np.percentile(housing_df[i], 99)
    

In [ ]:
housing_df[num_cols].describe([0,0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99,1]).T

- We can now see that the 99%, 100%tile and max are all the same eliminating the oultiers 

- Also pool area has zero as the mean so lets get rid of it 

In [ ]:
housing_df.drop('PoolArea',axis = 1, inplace = True)
num_cols = housing_df.select_dtypes(exclude='object').columns
num_cols

In [ ]:
#categorical columns 

cat_cols = housing_df.select_dtypes(include='object').columns
cat_cols

In [ ]:
#here we want to print out the value count in eah categorical column

# normalize used to get the percentage 
for i in list(cat_cols):
    print(housing_df[i].value_counts(normalize=True))
    print('**********************************')
    

Look through the value count and if you see maybe two categories or few where theres big disparity 
delete that because it wont have a big influence on price 

In [ ]:
drop_cols = ['Street','Utilities','LandSlope','Condition2','RoofMatl','Heating','CentralAir','Electrical','Functional',
             'PavedDrive','GarageCond','GarageQual','LandContour']

In [ ]:
housing_df.drop(columns=drop_cols, inplace=True)

In [ ]:
cat_cols = housing_df.select_dtypes(include='object').columns

In [ ]:
cat_cols

In [ ]:
housing_df.MSZoning = housing_df.MSZoning.apply(lambda x: 'other' if x not in ('RL','RM') else x)
housing_df.LotShape = housing_df.LotShape.apply(lambda x: 'other' if x not in ('Reg','IR1') else x)
housing_df.LotConfig = housing_df.LotConfig.apply(lambda x: 'other' if x not in ('Inside','Corner') else x)
housing_df.Condition1 = housing_df.Condition1.apply(lambda x: 'other' if x not in ('Norm') else x)
housing_df.BldgType = housing_df.BldgType.apply(lambda x: 'other' if x not in ('1Fam') else x)
housing_df.HouseStyle = housing_df.HouseStyle.apply(lambda x: 'other' if x not in ('1Story','2Story','1.5Fin') else x)
housing_df.RoofStyle = housing_df.RoofStyle.apply(lambda x: 'other' if x not in ('Gable','Hip','1.5Fin') else x)
housing_df.RoofStyle = housing_df.RoofStyle.apply(lambda x: 'other' if x not in ('Gable','Hip','1.5Fin') else x)
housing_df.RoofStyle = housing_df.RoofStyle.apply(lambda x: 'other' if x not in ('Gable','Hip','1.5Fin') else x)
housing_df.Exterior1st = housing_df.Exterior1st.apply(lambda x: 'other' if x not in ('VinylSd','HdBoard','MetalSd','Wd Sdng') else x)
housing_df.Exterior2nd = housing_df.Exterior2nd.apply(lambda x: 'other' if x not in ('VinylSd','HdBoard','MetalSd','Wd Sdng') else x)
housing_df.MasVnrType = housing_df.MasVnrType.apply(lambda x: 'other' if x not in ('None','none','BrkFace') else x)
housing_df.ExterQual = housing_df.ExterQual.apply(lambda x: 'other' if x not in ('TA','Gd') else x)
housing_df.ExterCond = housing_df.ExterCond.apply(lambda x: 'other' if x not in ('TA','Gd') else x)
housing_df.Foundation = housing_df.Foundation.apply(lambda x: 'other' if x not in ('PConc','CBlock','BrkTil') else x)
housing_df.BsmtQual = housing_df.BsmtQual.apply(lambda x: 'other' if x not in ('TA','Gd') else x)
housing_df.BsmtQual = housing_df.BsmtQual.apply(lambda x: 'other' if x not in ('TA','Gd') else x)
housing_df.BsmtExposure = housing_df.BsmtExposure.apply(lambda x: 'other' if x not in ('No','Av') else x)
housing_df.BsmtFinType1 = housing_df.BsmtFinType1.apply(lambda x: 'other' if x not in ('Unf','GLQ','ALQ','BLQ') else x)
housing_df.BsmtFinType2 = housing_df.BsmtFinType2.apply(lambda x: 'other' if x not in ('Unf') else x)
housing_df.HeatingQC = housing_df.HeatingQC.apply(lambda x: 'other' if x not in ('Ex','TA') else x)
housing_df.KitchenQual = housing_df.KitchenQual.apply(lambda x: 'other' if x not in ('TA','Gd') else x)
housing_df.GarageType = housing_df.GarageType.apply(lambda x: 'other' if x not in ('Attchd','Detchd') else x)
housing_df.GarageType = housing_df.GarageType.apply(lambda x: 'other' if x not in ('Attchd','Detchd') else x)
housing_df.SaleType = housing_df.SaleType.apply(lambda x: 'other' if x not in ('WD') else x)
housing_df.SaleCondition = housing_df.SaleCondition.apply(lambda x: 'other' if x not in ('Normal') else x)

In [ ]:
def year_to_cat(x):
    if x==0:
        return 'None'
    elif x < 1930:
        return 'lt_1930'
    elif x > 1930 and x <= 1960:
        return 'btw_30_60'
    elif x > 1960 and x <= 1990:
        return 'btw_60_90'
    elif x > 1990:
        return 'gt_90'


In [ ]:
housing_df['GarageYrBlt'] = housing_df['GarageYrBlt'].apply(lambda x: year_to_cat(x))

In [ ]:
cat_cols = housing_df.select_dtypes(include='object').columns
num_cols = housing_df.select_dtypes(exclude='object').columns

In [ ]:
num_cols

In [ ]:
plt.figure(figsize=(22,28))
for i,j in zip(num_cols[:-1], range(len(num_cols[:-1]))):
    plt.subplot(8,5,j+1)
    sns.scatterplot(data=housing_df, x='SalePrice', y=i)
plt.show()

In [ ]:
plt.figure(figsize=(22,30))
for i,j in zip(cat_cols, range(len(cat_cols))):
    plt.subplot(8,5,j+1)
    plt.xticks(rotation=45)
    sns.boxplot(data=housing_df, y='SalePrice', x=i)
plt.tight_layout()

In [ ]:
plt.figure(figsize=(22,22))
sns.heatmap(housing_df.corr(),cmap='RdYlGn', annot=True)

In [ ]:
housing_df.drop(columns='Neighborhood',inplace=True)
housing_df.reset_index(drop=True,inplace=True)

In [ ]:
cat_cols = housing_df.select_dtypes(include='object').columns

In [ ]:
print(num_cols,cat_cols)

In [ ]:
#lets create a new data frame with only dummmy categorical data and num columns 

dummies_df = housing_df[num_cols]

for i in cat_cols:
    temp = pd.get_dummies(housing_df[i],drop_first=True)
    dummies_df = pd.concat([dummies_df,temp],axis=1)

dummies_df.head()

In [ ]:
dummies_df.shape

## Lets split our data , Train our linear model , predict house sales and Evaluate model

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = dummies_df.drop(columns='SalePrice')

In [ ]:
y = dummies_df['SalePrice']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm = LinearRegression()

In [ ]:
lm.fit(X_train,y_train)

In [ ]:
#PREDICTIONS 
predictions = lm.predict(X_test)

In [ ]:
plt.figure(dpi=70)
g=sns.scatterplot(x=y_test,y=predictions)
g.set_xlabel('y_test',fontdict={'size':15})
g.set_ylabel('Predictions',fontdict={'size':15})

In [ ]:
sns.displot(y_test-predictions,bins=100,kde=True,)

In [ ]:
from sklearn import metrics

MAE = metrics.mean_absolute_error(y_test,predictions)

MSE = metrics.mean_squared_error(y_test,predictions)

RMSE = np.sqrt(MSE)

R2 = metrics.r2_score(y_test,predictions)

print(f'MAE: {MAE}')
print(f'MSE: {MSE}')
print(f'RMSE: {RMSE}')
print(f'R^2: {R2}')

Average Saleprice = $182000

In [ ]:
cdf = pd.DataFrame(data=lm.coef_,index=X.columns,columns=['Coefficients'])

In [ ]:
cdf

In [ ]:
dummies_df.shape

In [ ]:
housing_df.head()

In [ ]:
num_cols = num_cols.drop('SalePrice')

In [ ]:
#try some PCA PRINCIPLE CONCEPT 
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(housing_df[num_cols])

In [ ]:
scaled_numdata = scaler.transform(housing_df[num_cols])

In [ ]:
scaled_numdata

In [ ]:
#Apply PCA to numcols scaled data 
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)

In [ ]:
pca.fit(scaled_numdata)

In [ ]:
num_pca = pca.transform(scaled_numdata)

In [ ]:
num_pca

In [ ]:
num_pca.shape

In [ ]:
pd.DataFrame(pca.components_,columns=num_cols)

In [ ]:
num_cols

In [ ]:
dummies_df2 = pd.DataFrame(num_pca,columns=['component1','component2'])

for i in cat_cols:
    temp = pd.get_dummies(housing_df[i],drop_first=True)
    dummies_df2 = pd.concat([dummies_df2,temp],axis=1)

dummies_df2.head()

In [ ]:
X = dummies_df2
y = dummies_df['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
lmodel2 = LinearRegression()

In [ ]:
lmodel2.fit(X_train,y_train)

In [ ]:
pca_predictions = lmodel2.predict(X_test)

In [ ]:
plt.figure(dpi=70)
g=sns.scatterplot(x=y_test,y=pca_predictions)
g.set_xlabel('y_test',fontdict={'size':15})
g.set_ylabel('pca_predictions',fontdict={'size':15})

In [ ]:
#DISTRIBUTION GRAPH OF RESIDUALS 
sns.displot(y_test-pca_predictions,bins=100,kde=True)
plt.ylim(0.0,50.0)

In [ ]:
MAE = metrics.mean_absolute_error(y_test,pca_predictions)

MSE = metrics.mean_squared_error(y_test,pca_predictions)

RMSE = np.sqrt(MSE)

R2 = metrics.r2_score(y_test,pca_predictions)

print(f'MAE: {MAE}')
print(f'MSE: {MSE}')
print(f'RMSE: {RMSE}')
print(f'R^2: {R2}')

#### Using Principle component analysis to compress our numerical columns to two components.This improved our model slightly reducing error and rsquared values.

In [ ]:
len(X_test)

### USING NEURAL NETWORKS TO PREDICT PRICE

In [ ]:
X = dummies_df.drop(columns='SalePrice')
y = dummies_df['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=101)
    

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
#fit scaler to training data
#calcualtes the parameters needed for scaling , hence calculating std dev, min and max 
#we do it to training set only to avoid data leakage.
scaler.fit(X_train)

In [ ]:
#transform training data and test data 
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

### A commonly used "rule of thumb" for deep neural networks is the "sandwich rule.

In [ ]:
model = Sequential()
#layers in neural network 3 hidden layers
model.add(Dense(4,activation='relu'))
model.add(Dense(4,activation='relu'))
model.add(Dense(4,activation='relu'))
model.add(Dense(4,activation='relu'))
#output layer one nueron bacoause of what we are trying to predict
#no activation function because we want to predict price
model.add(Dense(1))

In [ ]:
#optimize the models gradient descent using the optimizer argument for example the ADAM
model.compile(optimizer='rmsprop',loss='mae')

In [ ]:
model.fit(X_train,y=y_train,epochs=400,verbose=1)

In [ ]:
loss_df = pd.DataFrame(model.history.history)
#to determine the amount of epochs needed for the minimum amount of loss
loss_df.plot()

In [ ]:
#returns the models loss based on the test set
#we used mean squared error
model.evaluate(X_test,y_test, verbose=0)

In [ ]:
#loss on the training set 
model.evaluate(X_train,y_train, verbose=0)

In [ ]:
test_predictions = model.predict(X_test)

In [ ]:
test_predictions = pd.DataFrame(test_predictions)

pred_df = pd.DataFrame(y_test.values,columns=['Test True Y'])

pred_df = pd.concat([pred_df,test_predictions],axis=1)

pred_df.columns = ['Test True Y', 'Model Predictions']

pred_df.head(10)

In [ ]:
##lets plot the predictions to the original y values
sns.scatterplot(x='Test True Y', y= 'Model Predictions',data=pred_df)

In [ ]:
MAE = metrics.mean_absolute_error(y_test,test_predictions)

MSE = metrics.mean_squared_error(y_test,test_predictions)

RMSE = np.sqrt(MSE)

R2 = metrics.r2_score(y_test,test_predictions)

print(f'MAE: {MAE}')
print(f'MSE: {MSE}')
print(f'RMSE: {RMSE}')
print(f'R^2: {R2}')

- We still see that the Linear regression model perfroms best at predicting the house prices with the lowest error, Although models can be fine tuned to perform better

### THE END 